# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [3]:

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split
# from sklearn import metrics
# from sklearn.ensemble import RandomForestClassifier

# from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy


In [29]:
df = pd.read_csv('previsao_de_renda.csv')

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.
    

In [33]:
df.columns

Index(['Unnamed: 0', 'data_ref', 'index', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'mau', 'renda'],
      dtype='object')

In [35]:
#1
df = df.drop(columns=['Unnamed: 0','data_ref', 'index'])

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   15000 non-null  object 
 1   posse_de_veiculo       15000 non-null  bool   
 2   posse_de_imovel        15000 non-null  bool   
 3   qtd_filhos             15000 non-null  int64  
 4   tipo_renda             15000 non-null  object 
 5   educacao               15000 non-null  object 
 6   estado_civil           15000 non-null  object 
 7   tipo_residencia        15000 non-null  object 
 8   idade                  15000 non-null  int64  
 9   tempo_emprego          12466 non-null  float64
 10  qt_pessoas_residencia  15000 non-null  float64
 11  mau                    15000 non-null  bool   
 12  renda                  15000 non-null  float64
dtypes: bool(3), float64(3), int64(2), object(5)
memory usage: 1.2+ MB


In [49]:
#df.groupby(['qtd_filhos']).sum()

In [47]:
categoria_mais_frequente = df['qtd_filhos'].value_counts().idxmax()
categoria_mais_frequente

0

In [63]:
categoria_mais_frequente2 = df['qt_pessoas_residencia'].value_counts().idxmax()
categoria_mais_frequente2

2.0

In [71]:
categoria_mais_frequente3 = df['tipo_renda'].value_counts()
categoria_mais_frequente3

tipo_renda
Assalariado         7826
Empresário          3311
Pensionista         2543
Servidor público    1316
Bolsista               4
Name: count, dtype: int64

In [91]:
df1 = smf.ols(' np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + C(qtd_filhos, Treatment(7)) + tipo_renda + idade + tempo_emprego + renda', data = df).fit()
df1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.715
Model:                            OLS   Adj. R-squared:                  0.715
Method:                 Least Squares   F-statistic:                     1837.
Date:                Wed, 24 Jul 2024   Prob (F-statistic):               0.00
Time:                        16:27:24   Log-Likelihood:                -7105.1
No. Observations:               12466   AIC:                         1.425e+04
Df Residuals:                   12448   BIC:                         1.438e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             6.5044      0.303     21.452      0.000       5.910       7.099
sexo[T.M]                             0.1757      0.009     19.721      0.000       0.158       0.193
posse_de_veiculo[T.True]              0.0131      0.008      1.570      0.116      -0.003       0.029
posse_de_imovel[T.True]               0.0448      0.008      5.514      0.000       0.029       0.061
C(qtd_filhos, Treatment(7))[T.0]      0.7778      0.303      2.568      0.010       0.184       1.372
C(qtd_filhos, Treatment(7))[T.1]      0.7731      0.303      2.552      0.011       0.179       1.367
C(qtd_filhos, Treatment(7))[T.2]      0.8011      0.303      2.643      0.008       0.207       1.395
C(qtd_filhos, Treatment(7))[T.3]      0.8260      0.305      2.711      0.007       0.229       1.423
C(qtd_filhos, Treatment(7))[T.4]      0.8977      0.318      2.819      0.005       0.274       1.522
C(qtd_filhos, Treatment(7))[T.5]      0.8188      0.428      1.912      0.056      -0.021       1.658
C(qtd_filhos, Treatment(7))[T.14]     0.8292      0.371      2.235      0.025       0.102       1.557
tipo_renda[T.Bolsista]                0.0749      0.214      0.350      0.727      -0.345       0.495
tipo_renda[T.Empresário]              0.0490      0.009      5.464      0.000       0.031       0.067
tipo_renda[T.Pensionista]             0.1725      0.143      1.208      0.227      -0.108       0.453
tipo_renda[T.Servidor público]        0.0417      0.013      3.232      0.001       0.016       0.067
idade                                 0.0014      0.000      2.998      0.003       0.000       0.002
tempo_emprego                         0.0098      0.001     14.731      0.000       0.008       0.011
renda                                 0.0001   8.89e-07    144.440      0.000       0.000       0.000
==============================================================================
Omnibus:                     7648.007   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           196698.837
Skew:                          -2.498   Prob(JB):                         0.00
Kurtosis:                      21.808   Cond. No.                     1.56e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.56e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Após ajustes o modelo acima foi o melhor encontrado observando o p-value e os R-quadrado e R-quadrado ajustado 